## imports

In [1]:
import os
import sys
sys.path.append("../")

import math

import pandas as pd
import numpy as np
import datetime as dt
import re

import pymc3 as pm

import altair as alt
alt.data_transformers.disable_max_rows()

from src.preprocess.preprocess import load_data, split_last
from src.plot.altair import plot_decompose, plot_all
from src.forecast.forecast import determine_estimates_minmax, forecast

In [8]:
boxid = [
    "ESD.000088-1",
    "063.623-1",
    "VRY.CHOPS-1",
    "HVT.111153-1",
    "TTR.251049-1",
    "BGL.CROLA-1",
][0]

## load data

In [10]:
%%time
df_data, df_meta = load_data(boxid=boxid)
plot_all(df_data=df_data)

[Fri, 22 Oct 2021 14:34:46] INFO [snowflake.py.read_meta] reading meta preprocess


INFO:SPARK:reading meta preprocess


[Fri, 22 Oct 2021 14:34:46] INFO [snowflake.py.read_meta] for boxids: ESD.000088-1


INFO:SPARK:for boxids: ESD.000088-1


[Fri, 22 Oct 2021 14:34:47] INFO [snowflake.py.read_week_extremes] reading extremes table


...

INFO:SPARK:removing leading low values (<0.01)


[Fri, 22 Oct 2021 14:34:49] INFO [preprocess.py.too_short] checking number of preprocess points (<=104)


INFO:SPARK:checking number of preprocess points (<=104)


CPU times: user 686 ms, sys: 50.3 ms, total: 737 ms
Wall time: 3.59 s


alt.LayerChart(...)

## split in train + test

In [ ]:
df_train, df_test = split_last(df_data.copy())

## estimate parameters + forecast

In [ ]:
%%time
df_estimates = determine_estimates_minmax(df_train)
df_total=pd.concat([df_data, df_estimates], axis=0)

## plot results

In [ ]:
plot_all(df_data=df_total, df_meta=df_meta)

In [ ]:
plot_decompose(df=df_total.query("extreme == 'max'"))